<a href="https://colab.research.google.com/github/sidhant0720/Sentiment_Analysis_NLP/blob/master/NLP_sentiment_analysis_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random     
import numpy as np

import pdb
from nltk.corpus import stopwords, twitter_samples
import pandas as pd
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [2]:
# downloads sample twitter dataset.
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [3]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [4]:
print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))

print('\nThe type of all_positive_tweets is: ', type(all_positive_tweets))
print('The type of a tweet entry is: ', type(all_negative_tweets[0]))


Number of positive tweets:  5000
Number of negative tweets:  5000

The type of all_positive_tweets is:  <class 'list'>
The type of a tweet entry is:  <class 'str'>


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [7]:
def RemoveLinksHash(t):
  t_cleaned = re.sub(r'^RT[\s]+', '', t)
  t_cleaned = re.sub(r'https?:\/\/.*[\r\n]*', '', t_cleaned)
  t_cleaned = re.sub(r'#', '', t_cleaned)
  return(t_cleaned)

In [8]:
def TokenizeTweet(t):
  tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
  t_tokens=tokenizer.tokenize(t)
  return t_tokens

In [9]:
stopwords_english=stopwords.words('english')
print(stopwords_english)
print(string.punctuation)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
def RemoveStopWordsPunctuations(t):
  t_updated=[]
  for word in t:
    if(word not in stopwords_english and word not in string.punctuation):
      t_updated.append(word)
  return t_updated  

In [11]:
def StemTweet(t):
  stemmer = PorterStemmer()
  tweet_stem=[]

  for word in t:
    stem_word=stemmer.stem(word)
    tweet_stem.append(stem_word)
  return tweet_stem

In [12]:
def processTweet(t):
  processedTweets=[]

  t2=RemoveLinksHash(t)
  t2=TokenizeTweet(t2)
  t2=RemoveStopWordsPunctuations(t2)
  t2=StemTweet(t2)
    
    #print(processedTweets)

  return t2

In [31]:
def count_tweets(result, tweets, ys):

    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    yslist = np.squeeze(ys).tolist()
    
    for y, tweet in zip(yslist, tweets):
      for word in processTweet(tweet):
          pair = (word,y)
          if pair in result:
              result[pair] += 1
          else:
              result[pair] = 1
    return result

In [22]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [32]:
freqs = count_tweets({}, train_x, train_y)

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11345


In [16]:
vocab = set(key[0] for key in freqs.keys())
print(len(vocab))


9088


In [25]:

result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('sad', 0): 1, ('tire', 0): 2, ('trick', 0): 1}

In [37]:
def lookup(freqs,word,y):
  return freqs.get((word,y),1)
  

In [43]:
def train_naive_bayes(freqs, train_x, train_y):
    
    loglikelihood = {}
    logprior = 0
    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    N_pos = N_neg = 0
    for pair in freqs.keys():
       
        if pair[1] > 0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]

    D = len(train_y)
    D_pos = sum(train_y)
    D_neg = float(D) - D_pos
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary
    for word in vocab:
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)
        p_w_pos = ( freq_pos + 1.0 )/(N_pos + V)
        p_w_neg = ( freq_neg + 1.0 )/(N_neg + V)
        loglikelihood[word] = float(logprior + np.log((p_w_pos)/(p_w_neg)))

    return float(logprior), loglikelihood

In [44]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9088


In [55]:
print(loglikelihood['printer'])

0.41104311033272856


In [49]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
 
    word_l = processTweet(tweet)
    p = 0
    p += logprior

    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p


In [51]:
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print( p)

1.5741939201384094


In [82]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):

    accuracy = 0  
    y_hats = []
    for tweet in test_x:
      
      if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
          y_hat_i = 1
      else:
          y_hat_i = 0

      y_hats.append(y_hat_i)
    #print(np.squeeze(test_y))
    error = (np.sum(np.absolute(np.subtract(np.asarray(y_hats),np.squeeze(test_y)))))/len(test_y)
    
    accuracy = 1 - error
    return accuracy

In [83]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

[1. 1. 1. ... 0. 0. 0.]
Naive Bayes accuracy = 0.9945


In [85]:
# Some error analysis
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            processTweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
1	0.00	b''
1	0.00	b'truli later move know queen bee upward bound movingonup'
1	0.00	b"woke feel incred sick idk caus drank starbuck 11 o'clock last night reaction med :)"
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	0.00	b''
1	0.00	b''
1	0.00	b'park get sunlight'
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	1.00	b'u prob fun david'
0	1.00	b'pat jay'
0	1.00	b'whatev stil l young >:-('


In [88]:
my_tweet = 'I am happy because'

p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

2.1487353582905078
